In [1]:
from natasha import NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
from natasha import Doc, Segmenter
import json
import pandas as pd
import re

In [2]:
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()

In [3]:
embedding = NewsEmbedding()
segmenter = Segmenter()

In [4]:
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embedding)

In [5]:
with open("../data/corpus_as_dict.json") as f:
    docs = json.load(f)

Создадим датасет с документами

In [6]:
titles = list(docs.keys())
texts = list(docs.values())
df = pd.DataFrame.from_dict({'title':titles, 'text':texts})

In [7]:
df.sample(5)

,title,text
126,Контрольный номер Библиотеки Конгресса,Контрольный номер Библиотеки Конгресса — контр...
71,Движитель,"Движитель — устройство, преобразующее энергию ..."
99,Интендант,Интендант — во Франции первоначально всякое ли...
294,Телефон,Телефон — аппарат для передачи и приема звука ...
20,Белфаст,"Белфаст — город в Великобритании, столица Севе..."


In [8]:
def get_ner(transcript):
  script = Doc(re.sub(r'\((.*?)\)', "", transcript))
  script.segment(segmenter)
  script.tag_morph(morph_tagger)
  for token in script.tokens:
    token.lemmatize(morph_vocab)
  script.tag_ner(ner_tagger)
  for span in script.spans:
    span.normalize(morph_vocab)
  named_ents = [(i.text, i.type, i.normal) for i in script.spans]
  normed_ents = []
  for word, tag, norm in named_ents:
    if len(word.split()) == 1 and tag == "LOC":
      for gram in range(len(analyzer.parse(word))):
        if "Geox" in analyzer.parse(word)[gram].tag:
          normed_ents.append((analyzer.parse(word)[gram].normal_form))
          break
        elif gram == len(analyzer.parse(word)) - 1:
          normed_ents.append((norm.lower().strip(".,!?;-")))
    else:
      normed_ents.append((norm.lower().strip(".,!?;-")))
  return sorted(normed_ents)

Следующая функция работает таким образом:

- текст сегментируется на токены
- на токены накладывается частеречная разметка
- токены лемматизируются
- выделяются именованные сущности
- токены нормализуются
- обрабатывается нормализация географических объектов, т.к. в процессе выяснилось, что natasha хуже их нормализует, чем pymorphy2.

In [9]:
df["named_entities"] = df.apply(lambda row: get_ner(row["text"]), axis=1)

In [10]:
df.sample(3)

,title,text,named_entities
280,Спасательная шлюпка,"Спасательная шлюпка — корабельная шлюпка, обыч...",[]
192,Норвегия,"Норвегия, официальное название — Королевство Н...","[антарктика, атлантический океан, буве, заморс..."
9,Англиканская церковь,Англиканство — одно из направлений христианств...,"[австралия, английская церковь, англиканская ц..."


In [11]:
has_ner = [i for i in df.index.values if df.named_entities[i]]

In [12]:
len(has_ner), df.shape[0]

(223, 363)

В 223 документах из 363 есть Именнованные сущности

In [13]:
df_ner = df[df.index.isin(has_ner)]

Будем работать именно с теми документами, где они есть

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

In [15]:
ner_voc = []
for row in df_ner.named_entities.tolist():
  ner_voc.extend(row)
len(ner_voc), len(set(ner_voc))

(1758, 955)

Всего в корпусе 1758 сущностей, уникальных 955

In [16]:
vocabulary = sorted(set(ner_voc))
corpus = df_ner.named_entities.apply(str).tolist()

In [17]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=18, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)

In [18]:
km.fit(X)

/home/alexanderlakiza/.virtualenvs/cs224/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "


KMeans(algorithm='full', max_iter=600, n_clusters=18, precompute_distances=True)

Есть два вида метрик оценки качества кластеризации:

внешние - используют дополнительные знания о кластерах (распределение по кластерам, количество кластеров)
внутренние - оценивают качество полученной структуры, не используя внешнюю информацию
Т.к. у нас нет никаких дополнительных знаний (но для меньшего объема данных их можно получить экспертно при необходимости), посмотрим на внутренние метрики. В sklearn есть следующие:

In [19]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.10245716239549112
1.4617191188377825


силуэт: показывает, насколько объект похож на свой кластер относительно других кластеров; если значение стремится к 1 - хорошее разбиение, если к -1 - плохое, если в районе 0 - кластеры пересекаются
индекс Дэвиcа-Болдуина: оценивает расстояние от объекта кластера до центроида и расстояние между центроидами; чем ниже, тем лучше разбиение

Оценка первого параметра при разном кол-ве кластеров колеблется от 0.7 до 1, что не очень хорошо  
А вот второй параметр уменьшается по мере увеличения количества кластеров, и имеет довольно неплохой результат

In [20]:
df_ner["label"] = km.predict(X)

<ipython-input-20-20dc8af91091>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ner["label"] = km.predict(X)


In [21]:
df_ner["label"].value_counts()

2     167
9      10
4       8
8       5
10      4
3       4
13      4
15      3
11      3
17      3
7       2
12      2
14      2
16      2
1       1
6       1
5       1
0       1
Name: label, dtype: int64

In [26]:
df_ner.query("label == 4")

,title,text,named_entities,label
15,Атм,Атмосфера — внесистемная единица измерения дав...,"[земля, международная организация законодатель...",4
51,Водород,Водород — химический элемент периодической сис...,[земля],4
93,Земля,Земля — третья по удаленности от Солнца планет...,"[арктика, венера, земля, земля, земля, земля, ...",4
124,Компас,Компас; на профессиональном жаргоне моряков — ...,[земля],4
150,Луна,Луна — единственный естественный спутник Земли...,"[земля, земля, земля, земля, земля, луна, луна...",4
152,Магнитная девиация,Магнитная девиация или магнитное отклонение — ...,[земля],4
316,Углекислый газ,Диоксид углерода или двуокись углерода — бесцв...,"[земля, земля, земля, земля, земля, земля, сол...",4
331,Человек,"Человек — общественное существо, обладающее ра...","[в. г. борзенков, земля, и. т. фролов]",4


In [29]:
df_ner.query("label == 13")

,title,text,named_entities,label
148,Лошадиная сила,Лошадиная сила — внесистемная единица мощности...,"[англия, международная система, россия, россия...",13
168,Моллюск,"Моллюски, или мягкотелые, — тип первичноротых ...",[россия],13
252,Рыбы,Рыбы — парафилетическая группа водных позвоноч...,[россия],13
256,Санкт-Петербург,Санкт-Петербург — второй по численности населе...,"[военно-морской флот, вооруженные силы, гераль...",13


In [33]:
df_ner.query("label==1|label==0|label==5|label==6")

,title,text,named_entities,label
135,Лаган,Лаган — река в Северной Ирландии. Берет начало...,"[антрим, белфаст, белфаст, даун, ирландское мо...",6
240,Ренессанс,"Возрождение, или Ренессанс — имеющая мировое з...","[джорджо вазари, европа, жюль мишле, италия, р...",1
245,Роберт Баллард,Роберт Дуэйн Баллард — американский исследоват...,"[nautilus, uss yorktown, вмс, джон кеннеди, ро...",0
315,Убийство Джона Кеннеди,Убийство 35-го президента США Джона Кеннеди бы...,"[верховный суд, даллас, джон кеннеди, джон кен...",5


In [24]:
df_ner.query("label == 12")

,title,text,named_entities,label
136,Лас-Вегас,"Лас-Вегас — город на западе США, в штате Невад...","[даунтаун, кларк, лас-вегас, лас-вегас, лас-ве...",12
149,Луксор Лас-Вегас,"«Луксор Лас-Вегас» — гостиница и казино, распо...","[кларк, лас-вегас-стрип, луксор лас-вегас, нев...",12


In [25]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(18):
  print("Cluster %d:" % i, end='')
  for ind in order_centroids[i, :10]:
    print(' %s' % terms[ind], end='')
  print()

Cluster 0: сша nautilus вмс японское море ж. б. ламарком заморская территория залив белфаст зал слава закавказье жюль мишле
Cluster 1: ренессанс италия европа японское море ж. б. ламарком заморская территория залив белфаст зал слава закавказье жюль мишле
Cluster 2: канада великобритания англия атлантика северная франция атлантический сша европа россия
Cluster 3: франция париж сена республика ришелье европа нато паризии сите версаль
Cluster 4: земля луна российская солнце марс тейи венера арктика меркурий жан кальвин
Cluster 5: уоррен сша даллас жаклин ли ехидну залив белфаст зал слава закавказье жюль мишле
Cluster 6: лисберн лаган белфаст антрим даун ирландия северная закавказье женева жоау фернандеша лэврадур
Cluster 7: конгресс сша японское море ехидну заморская территория залив белфаст зал слава закавказье жюль мишле жюль ардуэн-мансар
Cluster 8: сша америка штат бифштекс британия bloomberg swift россия северная центральная
Cluster 9: великобритания ирландия лондон англия северная ш

### Вывод
В общем и целом, разбиение на кластеры выглядит успешным, так как примерно прослеживается тематика почти каждого кластера